In [1]:
import sklearn.model_selection
import pandas as pd
import numpy as np
import math

from hw3_traitors import *

blotter = pd.read_csv('blotter.csv')
print(blotter)

      trade_id     date asset   trip action type     price     status
0            1   1/3/20   IVV  ENTER    BUY  LMT  323.0600  SUBMITTED
1            1   1/3/20   IVV  ENTER    BUY  LMT  323.0600     FILLED
2            1   1/3/20   IVV   EXIT   SELL  LMT  326.2906  SUBMITTED
3            1   1/8/20   IVV   EXIT   SELL  LMT  326.2906     FILLED
4            2   1/6/20   IVV  ENTER    BUY  LMT  320.5700  SUBMITTED
...        ...      ...   ...    ...    ...  ...       ...        ...
2606       810  3/22/23   IVV  ENTER    BUY  LMT  398.2100  SUBMITTED
2607       810  3/22/23   IVV  ENTER    BUY  LMT  398.2100     FILLED
2608       810  3/22/23   IVV   EXIT   SELL  LMT  402.1921  SUBMITTED
2609       810  3/23/23   IVV   EXIT   SELL  LMT  402.1921       LIVE
2610       811  3/23/23   IVV  ENTER    BUY  LMT  391.5400       LIVE

[2611 rows x 8 columns]


In [2]:
ledger = blotter_to_ledger(blotter)
print(ledger)

/Users/benclayman/Documents/benjamin-clayman-533/HW3/hw3_traitors.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  blotter['date'] = pd.to_datetime(blotter['date'])


    trade_id asset   dt_enter    dt_exit success    n       rtn
0          1   IVV 2020-01-03 2020-01-08       1    6  0.001658
1          2   IVV 2020-01-06 2020-01-08       0    3  0.000000
2          3   IVV 2020-01-07 2020-01-09       0    3  0.000000
3          4   IVV 2020-01-08 2020-01-10       0    3  0.000000
4          5   IVV 2020-01-09 2020-01-13       0    5  0.000000
..       ...   ...        ...        ...     ...  ...       ...
806      807   IVV 2023-03-17 2023-03-21       1    5  0.001990
807      808   IVV 2023-03-20 2023-03-22       0    3  0.000000
808      809   IVV 2023-03-21        NaT     NaN  NaN       NaN
809      810   IVV 2023-03-22        NaT     NaN  NaN       NaN
810      811   IVV 2023-03-23        NaT     NaN  NaN       NaN

[811 rows x 7 columns]


In [3]:
# build your set of features here.
# merge them by date to add to this dataframe.
features = pd.read_csv('daily-treasury-rates.csv')
features['Date'] = pd.to_datetime(features['Date'])

vix = pd.read_csv('^VIX.csv')[['Date', 'Open']]
vix['Date'] = pd.to_datetime(vix['Date'])

features = features.merge(vix, on='Date')
features.head(10)

/var/folders/pp/nyzsgs116bd833r1yw537x5w0000gn/T/ipykernel_56532/4093259442.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  features['Date'] = pd.to_datetime(features['Date'])


,Date,1 Mo,2 Mo,3 Mo,4 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr,Open
0,2023-03-29,4.34,4.50,4.80,4.91,4.92,4.59,4.08,3.87,3.67,3.62,3.57,3.91,3.78,19.389999
1,2023-03-28,4.24,4.39,4.80,4.86,4.90,4.55,4.02,3.84,3.63,3.60,3.55,3.90,3.77,20.530001
2,2023-03-27,4.22,4.47,4.91,4.90,4.86,4.51,3.94,3.79,3.59,3.57,3.53,3.90,3.77,22.049999
3,2023-03-24,4.28,4.48,4.74,4.78,4.76,4.32,3.76,3.58,3.41,3.40,3.38,3.77,3.64,22.110001
4,2023-03-23,4.26,4.48,4.73,4.84,4.80,4.38,3.76,3.57,3.39,3.39,3.38,3.78,3.66,21.540001
5,2023-03-22,4.16,4.56,4.79,4.91,4.95,4.56,3.96,3.76,3.54,3.53,3.48,3.83,3.68,21.799999
6,2023-03-21,4.07,4.50,4.78,4.89,4.96,4.68,4.17,3.98,3.73,3.68,3.59,3.90,3.73,24.160000
7,2023-03-20,4.34,4.56,4.81,4.84,4.80,4.34,3.92,3.77,3.56,3.55,3.47,3.83,3.65,27.770000
8,2023-03-17,4.31,4.51,4.52,4.79,4.71,4.26,3.81,3.68,3.44,3.45,3.39,3.76,3.60,22.920000
9,2023-03-16,4.22,4.66,4.74,4.92,4.94,4.49,4.14,3.99,3.72,3.67,3.56,3.87,3.71,26.190001


In [14]:
import numpy as np
from sklearn.linear_model import Perceptron
from sklearn.preprocessing import StandardScaler

# Make a training set and let's try it out on two upcoming trades.
# Choose a subset of data:
X = features.drop('Date', axis=1).head(20)
x_test = features.drop('Date', axis=1).iloc[[20, 21]]
y = np.asarray(ledger.success.head(20), dtype="|S6")

print(X)
print(x_test)
print(y)

    1 Mo  2 Mo  3 Mo  4 Mo  6 Mo  1 Yr  2 Yr  3 Yr  5 Yr  7 Yr  10 Yr  20 Yr   
0   4.34  4.50  4.80  4.91  4.92  4.59  4.08  3.87  3.67  3.62   3.57   3.91  \
1   4.24  4.39  4.80  4.86  4.90  4.55  4.02  3.84  3.63  3.60   3.55   3.90   
2   4.22  4.47  4.91  4.90  4.86  4.51  3.94  3.79  3.59  3.57   3.53   3.90   
3   4.28  4.48  4.74  4.78  4.76  4.32  3.76  3.58  3.41  3.40   3.38   3.77   
4   4.26  4.48  4.73  4.84  4.80  4.38  3.76  3.57  3.39  3.39   3.38   3.78   
5   4.16  4.56  4.79  4.91  4.95  4.56  3.96  3.76  3.54  3.53   3.48   3.83   
6   4.07  4.50  4.78  4.89  4.96  4.68  4.17  3.98  3.73  3.68   3.59   3.90   
7   4.34  4.56  4.81  4.84  4.80  4.34  3.92  3.77  3.56  3.55   3.47   3.83   
8   4.31  4.51  4.52  4.79  4.71  4.26  3.81  3.68  3.44  3.45   3.39   3.76   
9   4.22  4.66  4.74  4.92  4.94  4.49  4.14  3.99  3.72  3.67   3.56   3.87   
10  4.23  4.56  4.75  4.85  4.73  4.19  3.93  3.83  3.59  3.57   3.51   3.82   
11  4.47  4.77  4.88  4.95  4.93  4.45  

In [15]:
sc = StandardScaler()

sc.fit(X)
X_std = sc.transform(X)
x_test_std = sc.transform(x_test)

print(X_std)


[[-0.46897463 -0.9298208  -0.39682656 -0.31145449 -0.3416057  -0.21722355
  -0.4751554  -0.54707139 -0.45550724 -0.50698712 -0.39819284 -0.19354466
   0.11326837 -1.08358718]
 [-0.85655698 -1.55730723 -0.39682656 -0.72126302 -0.43920732 -0.33385365
  -0.60744925 -0.62394325 -0.57657229 -0.57715834 -0.48619126 -0.26522786
   0.02265367 -0.64249052]
 [-0.93407345 -1.10095347  0.45076415 -0.39341619 -0.63441058 -0.45048375
  -0.78384105 -0.75206302 -0.69763734 -0.68241518 -0.57418968 -0.26522786
   0.02265367 -0.05436344]
 [-0.70152404 -1.04390924 -0.85914876 -1.37695667 -1.12241871 -1.00447671
  -1.18072259 -1.29016603 -1.24243006 -1.27887061 -1.23417781 -1.19710955
  -1.15533739 -0.0311471 ]
 [-0.77904051 -1.04390924 -0.93620246 -0.88518643 -0.92721546 -0.82953156
  -1.18072259 -1.31578998 -1.30296258 -1.31395623 -1.23417781 -1.12542634
  -0.97410799 -0.25169504]
 [-1.16662285 -0.58755548 -0.47388026 -0.31145449 -0.19520325 -0.30469613
  -0.7397431  -0.82893488 -0.84896865 -0.82275764 -

In [16]:
ppn = Perceptron(eta0=0.1)
ppn.fit(X_std, y)

y_pred = ppn.predict(x_test_std)
print(y_pred)

[b'1' b'1']


In [18]:
print(ledger.iloc[[20, 21]])

   trade_id asset   dt_enter    dt_exit success  n  rtn
20       21   IVV 2020-02-03 2020-02-05       0  3  0.0
21       22   IVV 2020-02-04 2020-02-06       0  3  0.0
